In [26]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio


In [6]:

load_dotenv(override=True)


True

In [11]:
attractions_planner_agent_instructions = """
You are a attractions planner. You are given a destination and need to plan all must-do things in that city.
You should give only the list of attractions and not suggestions on Restaurants, Hotels, etc.
Attractions should be something like historical places, museums, parks, etc.. This should not include any activities like hiking,skydiving, kayaking, sunset sails, snorkeling, etc.
"""

activity_planner_agent_instructions = """
You are a activity planner. You are given a destination 
You need to find all local best activities in that city with their popularity.
Activities should be something to experience like skydiving, kayaking, sunset sails, snorkeling, etc.
"""

restaurant_planner_agent_instructions = """
You are a restaurant planner. You are given a destination and number of days to spend.
You need to find all local best restaurants in that city with their ratings and reviews. 
You need to plan for 3 meals a day for the number of days you are staying.
"""

attractions_planner_agent = Agent(
    name="Attractions Planner",
    instructions = attractions_planner_agent_instructions,
)

activity_planner_agent = Agent(
    name="Activity Planner",
    instructions = activity_planner_agent_instructions,
)

restaurant_planner_agent = Agent(
    name="Restaurant Planner",
    instructions = restaurant_planner_agent_instructions,
)



In [16]:
itenary_planner_agent_instructions = """
    You are given a list of attractions, activities and restaurants. 
    You need to pick best attractions, activities and restaurants out of the list to plan a trip to the destination.
    You give a day by day plan of the trip with timings for various attractions and activities.
    You'll also plan for appropriate schedules for 3 meals a day in nearby restaurants to the current schedule.
"""

itenary_planner_agent = Agent(
    name="Itenary Planner",
    instructions=itenary_planner_agent_instructions,
)

In [33]:

# message = "SanDiego, 5 days"

with trace("Planning Itenary"):
    results = await asyncio.gather(
        Runner.run(attractions_planner_agent, message),
        Runner.run(activity_planner_agent, message),
        Runner.run(restaurant_planner_agent, message),
    )
    outputs = [result.final_output for result in results]

    todos = "Attractions, activities and restaurants:\n\n".join(outputs)

    plan = await Runner.run(itenary_planner_agent, todos)

    print(f"Best Plan:\n{plan.final_output}")



Best Plan:
Here is a detailed day-by-day itinerary for your San Diego trip, combining attractions, activities, and meals:

### Day 1: Balboa Park and Museums
- **7:30 AM:** Breakfast at *The Cottage La Jolla*  
  Start your day with delightful French toast and omelets.

- **9:00 AM:** Explore Balboa Park  
  Wander through gardens and enjoy the scenic beauty.

- **11:00 AM:** Visit San Diego Museum of Art  
  Appreciate world-class art collections.

- **1:00 PM:** Lunch at *Hodad's*  
  Enjoy famous burgers in a casual atmosphere.

- **2:30 PM:** San Diego Natural History Museum  
  Discover fascinating exhibits of natural history.

- **4:00 PM:** Museum of Man  
  Explore anthropology and cultural history.

- **6:30 PM:** Dinner at *Cucina Urbana*  
  Savor modern Italian cuisine with a great wine list.

### Day 2: History and Heritage
- **8:00 AM:** Breakfast at *Great Maple*  
  Enjoy maple bacon doughnuts and fresh breakfast dishes.

- **9:30 AM:** USS Midway Museum  
  Explore his

In [21]:
calender_generator_agent_instructions = """
You are a calender generator. You are given a ordered list of attractions, activities and restaurants with appropriate days and timings.
You need to generate a single ics file with all the events and their details.
You need to include all the details like name, description, start and end time, location, etc.
This ics file should be in a format that can be imported into any calendar app.
"""

calender_generator_agent = Agent(
    name="Calender Generator",
    instructions = calender_generator_agent_instructions,
)

In [28]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("harishjmhss@gmail.com")  # Change to your verified sender
    to_email = To("harishneyveli96@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

# send_email("This is a test email")

In [55]:
activity_planner_tool = activity_planner_agent.as_tool(tool_name="activity_planner_tool", tool_description="Plan only activities for the given destination and number of days")
attractions_planner_tool = attractions_planner_agent.as_tool(tool_name="attractions_planner_tool", tool_description="Plan only attractions for the given destination and number of days")
restaurant_planner_tool = restaurant_planner_agent.as_tool(tool_name="restaurant_planner_tool", tool_description="Plan only restaurants for the given destination and number of days")
itenary_planner_tool = itenary_planner_agent.as_tool(tool_name="itenary_planner_tool", tool_description="Plan a trip to the given destination and number of days by calling activity planner, attractions planner and restaurant planner tools")
calender_generator_tool = calender_generator_agent.as_tool(tool_name="calender_generator_tool", tool_description="Generate a calender with all the events and their details")


In [77]:
tools = [
    activity_planner_tool,
    attractions_planner_tool,
    restaurant_planner_tool,
    calender_generator_tool,
    send_email
]


In [78]:
travel_planner_agent_instructions = """
    You are a travel planner. You are given a particular destination and number of days to spend. 
    You need to use the tools given to you to plan the trip.
    You need to use the tools in a way that is most efficient and effective.
    Send email with the itenary to the user including the calender appointment as a downloadable ics file.
"""

travel_planner_agent = Agent(
    name="travel_planner_agent",
    instructions = travel_planner_agent_instructions,
    tools = tools,
    model = "gpt-4o-mini"
)



In [79]:
tools

[FunctionTool(name='activity_planner_tool', description='Plan only activities for the given destination and number of days', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'activity_planner_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fee524ba700>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='attractions_planner_tool', description='Plan only attractions for the given destination and number of days', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'attractions_planner_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fee524baf20>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='restaurant_planne

In [80]:
message = "SanDiego, 5 days"

with trace("Trip Planner"):
    result = await Runner.run(travel_planner_agent, message)

In [63]:
print(results)

[RunResult(input='SanDiego, 5 days', new_items=[MessageOutputItem(agent=Agent(name='Travel Planner', instructions='You are a attractions planner. You are given a destination and need to plan all must-do things in that city.\n    You should give only the list of attractions and not suggestions on Restaurants, Hotels, etc.\n    Attractions should be something like historical places, museums, parks, etc.. This should not include any activities like hiking,skydiving, kayaking, sunset sails, snorkeling, etc.\n    ', prompt=None, handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=Non